## Mini-Batch and Modified MiniBatch
### Imports

In [27]:
import numpy as np
import matplotlib as plt
import math
from random import*
from numpy.testing import assert_almost_equal

In [28]:
from collections import defaultdict

In [29]:
def GA(n,d):
    C = np.zeros((d,d)) # covariance matrix
    for i in range(d):
        for j in range(d):
            C[i][j] = 2*(0.5**abs(i-j))
    mean = np.ones(d)
    X = np.random.multivariate_normal(mean,C,(n)) # this is A
    beta = list()
    beta.extend(np.ones(10))
    beta.extend(0.1*np.ones(d-20))
    beta.extend(np.ones(10))
    beta = np.array(beta)
    noise = np.random.normal(loc=0.0,scale =9,size=n)
    y = X.dot(beta) + noise # this is b
    return X,y

In [30]:
def T3(n,d):
    df = 3
    C = np.zeros((d,d)) # covariance matrix
    for i in range(d):
        for j in range(d):
            C[i][j] = 2*(0.5**abs(i-j))
    mean = np.ones(d)
    x = np.random.chisquare(df, n)/df
    X = np.random.multivariate_normal(mean,C,(n))
    X = X/np.sqrt(x)[:,None]
    beta = list()
    beta.extend(np.ones(10))
    beta.extend(0.1*np.ones(d-20))
    beta.extend(np.ones(10))
    beta = np.array(beta)
    noise = np.random.normal(loc=0.0,scale =9,size=n)
    y = X.dot(beta) + noise # this is b
    return X,y

In [31]:
def T1(n,d):
    df = 1
    C = np.zeros((d,d)) # covariance matrix
    for i in range(d):
        for j in range(d):
            C[i][j] = 2*(0.5**abs(i-j))
    mean = np.ones(d)
    x = np.random.chisquare(df, n)/df
    X = np.random.multivariate_normal(mean,C,(n))
    X = X/np.sqrt(x)[:,None]
    beta = list()
    beta.extend(np.ones(10))
    beta.extend(0.1*np.ones(d-20))
    beta.extend(np.ones(10))
    beta = np.array(beta)
    noise = np.random.normal(loc=0.0,scale =9,size=n)
    y = X.dot(beta) + noise # this is b
    
    return X,y

# Given Inputs:
### b: batch size
### k: number of cluster
### t: number of iteration

In [32]:
batch_size = int(input('enter the batch size: '))
k = int(input('enter number of cluster required: '))
t = int(input('enter number of iterations: '))

In [33]:
n = int(input('# rows = '))
d = int(input('# columns = '))

In [92]:
def mini_batch(Cluster_centre):
    global Matrix
    global k,batch_size
    global mini_batch_cost
    v = np.zeros(Cluster_centre.shape[0])
    for i in range(10):
        totalCost_mini_batch=0
        rows = Matrix.shape[0]
        index = np.random.choice(rows,batch_size,replace=False)
        M = Matrix[index,:]
        dx = np.zeros(M.shape[0])# for storing the nearest cluster
        dist = np.full((M.shape[0]), np.inf)
        
        for point in range(M.shape[0]):
            for cc in range(k):
                eucd_dist = np.square(np.linalg.norm(M[point]-Cluster_centre[cc],ord=2))
                if(eucd_dist < dist[point]):
                    dx[point] = cc
                    dist[point]=eucd_dist
        #print(dx)
        for point in range(M.shape[0]):
            nearest_cluster = Cluster_centre[int(dx[point])]
            v[int(dx[point])] +=1
            n = 1/v[int(dx[point])]
            Cluster_centre[int(dx[point])] = Cluster_centre[int(dx[point])]*(1-n)  + np.multiply(M[point],n)
            
        for x in Matrix:
            min_dist = np.inf
            for c in Cluster_centre:
                eucd_dist = np.square(np.linalg.norm(x-c,ord=2))
                if(min_dist>eucd_dist):
                    min_dist=eucd_dist
            totalCost_mini_batch +=min_dist
        mini_batch_cost[i].append(totalCost_mini_batch)
        #print("Iteration",i)
        #print("Total Cost----",totalCost_mini_batch)

## Implementing modified Mini-Batch

In [93]:
def modified_MB(Cluster_centre):
    global Matrix,k,modified_mb_cost
    
    v = np.zeros(Cluster_centre.shape[0])
    prob = computeProb(Cluster_centre,Matrix)
    
    for i in range(10):
        totalCost_modified_mini_batch=0
        index = np.random.choice(Matrix.shape[0],batch_size,replace=False,p=prob)
        M = Matrix[index,:]
        dx = np.zeros(M.shape[0])# for storing the nearest cluster
        dist = np.full((M.shape[0]), np.inf)
        
        for point in range(M.shape[0]):
            for cc in range(k):
                eucd_dist = np.square(np.linalg.norm(M[point]-Cluster_centre[cc],ord=2))
                if(eucd_dist < dist[point]):
                    dx[point] = cc
                    dist[point]=eucd_dist
                    
        for point in range(M.shape[0]):
            nearest_cluster = Cluster_centre[int(dx[point])]
            v[int(dx[point])] +=1
            n = 1/v[int(dx[point])]
            Cluster_centre[int(dx[point])] = Cluster_centre[int(dx[point])]*(1-n)  + np.multiply(M[point],n)
            
        for x in Matrix:
            min_dist = np.inf
            for c in Cluster_centre:
                eucd_dist = np.square(np.linalg.norm(x-c,ord=2))
                if(min_dist>eucd_dist):
                    min_dist=eucd_dist
            totalCost_modified_mini_batch +=min_dist    
        modified_mb_cost[i].append(totalCost_modified_mini_batch)
        #print("Iteration",i)
        #print("Total Cost----",totalCost_modified_mini_batch)
        

In [36]:
def computeProb(CC,Data):
    prob = np.zeros(Data.shape[0])
    total_cost = 0
    for i in range(Data.shape[0]):
        min_dist=np.inf
        for c in CC:
            eucd_dist = np.square(np.linalg.norm(Data[i]-c,ord=2))
            if(min_dist>eucd_dist):
                min_dist=eucd_dist
        total_cost +=min_dist
        prob[i]=min_dist
    return prob/total_cost

### Utility Functions
#### printing for printing data
#### run_ to run for both the algos implemented

In [98]:
def printing(data):
    for i in range(t):
        #print("Iteration:",i)
        print(sum(data[i])/100)

In [99]:
def run_():
    global Matrix,mini_batch_cost,modified_mb_cost
    for l in range(100):
        indexes = np.random.choice(Matrix.shape[0],k)
        CC = Matrix[indexes,:] # cluster_centre
        mini_batch(CC)
        modified_MB(CC)
    printing(mini_batch_cost)
    print("$")
    print("----Modified Mini_Batch Begin")
    print("$")
    printing(modified_mb_cost)

### Data Near Uniform: GA

In [100]:
A,b = GA(n,d)
Matrix = np.c_[A,b]
mini_batch_cost=defaultdict(list) # mini_batch distionary of list
modified_mb_cost=defaultdict(list) # modified_mini_batch dictionary of list
run_()

69820.03572949566
62035.25569095423
58276.70477894798
56254.772449629236
54896.22938855754
53903.793707624245
53246.10489516998
52685.09595907712
52241.02466515771
51872.7936999876
$
----Modified Mini_Batch Begin
$
64395.615375343616
59492.358032105345
56622.7708718217
54807.67833546546
53578.086999797764
52721.77971206426
52120.535970435674
51615.2911535418
51206.38043337346
50907.743428655725


### Data Moderatey Uniform

In [101]:
A,b = T3(n,d)
Matrix = np.c_[A,b]
mini_batch_cost=defaultdict(list) # mini_batch distionary of list
modified_mb_cost=defaultdict(list) # modified_mini_batch dictionary of list
run_()

373970.2725574194
338761.4885193971
320498.4032100612
307718.3206512904
299603.4209715184
293997.22834357346
289258.81372532115
285502.09216413664
281696.08360582776
279094.340920616
$
----Modified Mini_Batch Begin
$
267663.1647266214
240660.21536463327
225456.18055306384
216146.4771409298
208642.61075100233
203697.80977459744
199870.04922532348
196595.20081522546
193998.68045401803
191876.31457941557


### Data Very Non-Uniform

In [102]:
A,b = T1(n,d)
Matrix = np.c_[A,b]
mini_batch_cost=defaultdict(list) # mini_batch distionary of list
modified_mb_cost=defaultdict(list) # modified_mini_batch dictionary of list
run_()

7357500295.741517
7316034930.508034
7188112433.643279
7008500544.006056
6842777992.855482
6821794041.249037
6755236508.555392
6635735617.170813
6617195124.903004
6577716882.58208
$
----Modified Mini_Batch Begin
$
4546922374.559704
3775814641.973122
3290882410.235468
2960813313.1239414
2720956680.3880806
2538424274.13589
2394772955.844936
2271967524.022173
2162627553.6947703
2063851361.6311743
